# 1. 提取GenImage CASIAv2数据集

In [1]:
import os
import json
from pathlib import Path

In [2]:
au_path = "/home/censoring/GenImage/ADM/train/nature"
tp_path = "/home/censoring/GenImage/ADM/train/ai"
json_list = []
img_exts = {".jpg", ".png", ".bmp", ".tif", ".tiff",".jpeg"}

In [3]:
for p in Path(au_path).glob("*"):
    if p.suffix.lower() in img_exts:
        json_list.append({
            "path": str(p),
            "label": 0
        })
for p in Path(tp_path).glob("*"):
    if p.suffix.lower() in img_exts:
        json_list.append({
            "path": str(p),
            "label": 1
        })
print(f"Total images: {len(json_list)}")


Total images: 319453


In [4]:
json_list[:3]

[{'path': '/home/censoring/GenImage/ADM/train/nature/n03457902_5852.JPEG',
  'label': 0},
 {'path': '/home/censoring/GenImage/ADM/train/nature/n04562935_25283.JPEG',
  'label': 0},
 {'path': '/home/censoring/GenImage/ADM/train/nature/n06359193_42475.JPEG',
  'label': 0}]

In [5]:
output_file = "GenImage_ADM.json"
with open(output_file, "w") as f:
    json.dump(json_list, f, indent=4)
print(f"JSON file saved to {output_file}")

JSON file saved to GenImage_ADM.json


# 2. 提取faceforensic数据集

In [11]:
import json
import os

In [12]:
ff_json = "/home/chengxiaozhen/Test/ForensicHub/DeepfakeBench/DeepfakeBench/preprocessing/dataset_json/FaceForensics++.json"

with open(ff_json,"r",encoding="utf-8") as f:
    ff_data = json.load(f)


In [13]:
base_dir = "/home/chengxiaozhen/Test/ForensicHub/DeepfakeBench/datasets/rgb"
def collect_frames(node):
    results = []
    if isinstance(node,dict):
        for k,v in node.items():
            if k == "frames" and isinstance(v,list):
                for frame in v:
                    results.append(os.path.join(base_dir,frame))
            else:
                results.extend(collect_frames(v))
    elif isinstance(node,list):
        for item in node:
            results.extend(collect_frames(item))
    return results


def extract_split(data,subset_key,split):
    root = data.get("FaceForensics++")
    if not isinstance(root,dict):
        raise KeyError("Invalid JSON structure: 'FaceForensics++' key not found or is not a dictionary.")
    subset = root.get(subset_key)
    split_dict = subset.get(split,{}) or {}
    return collect_frames(split_dict)

In [14]:
ff_real = extract_split(ff_data,subset_key="FF-real",split="train")


In [15]:
ff_real[:5]

['/home/chengxiaozhen/Test/ForensicHub/DeepfakeBench/datasets/rgb/FaceForensics++/original_sequences/youtube/c23/frames/001/000.png',
 '/home/chengxiaozhen/Test/ForensicHub/DeepfakeBench/datasets/rgb/FaceForensics++/original_sequences/youtube/c23/frames/001/014.png',
 '/home/chengxiaozhen/Test/ForensicHub/DeepfakeBench/datasets/rgb/FaceForensics++/original_sequences/youtube/c23/frames/001/029.png',
 '/home/chengxiaozhen/Test/ForensicHub/DeepfakeBench/datasets/rgb/FaceForensics++/original_sequences/youtube/c23/frames/001/044.png',
 '/home/chengxiaozhen/Test/ForensicHub/DeepfakeBench/datasets/rgb/FaceForensics++/original_sequences/youtube/c23/frames/001/059.png']

In [16]:
ff_df = extract_split(ff_data,subset_key="FF-DF",split="train")
ff_f2f = extract_split(ff_data,subset_key="FF-F2F",split="train")
ff_fs = extract_split(ff_data,subset_key="FF-FS",split="train")
ff_nt = extract_split(ff_data,subset_key="FF-NT",split="train")

In [17]:
ff_df[:2]

['/home/chengxiaozhen/Test/ForensicHub/DeepfakeBench/datasets/rgb/FaceForensics++/manipulated_sequences/Deepfakes/c23/frames/001_870/000.png',
 '/home/chengxiaozhen/Test/ForensicHub/DeepfakeBench/datasets/rgb/FaceForensics++/manipulated_sequences/Deepfakes/c23/frames/001_870/014.png']

In [20]:
json_output = []

for path in ff_real:
    json_output.append({
        "path": path,
        "label": 0
    })

for path in ff_df:
    json_output.append({
        "path": path,
        "label": 1
    })
for path in ff_f2f:
    json_output.append({
        "path": path,
        "label": 1
    })
for path in ff_fs:
    json_output.append({
        "path": path,
        "label": 1
    })
for path in ff_nt:
    json_output.append({
        "path": path,
        "label": 1
    })
output_file = "FaceForensics++.json"
with open(output_file, "w") as f:
    json.dump(json_output, f, indent=4)
print(f"JSON file saved to {output_file}")

JSON file saved to FaceForensics++.json
